In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn import tree, ensemble
import xgboost as xgb
import os
import time
from joblib import dump, load
from sklearn.metrics import confusion_matrix


Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/envs/bigdatalab_cpu_202101/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
status_path = '../../Data/filtered_status.csv'
stations_path = '../../Data/station.csv'

status_df = pd.read_csv(status_path, parse_dates=['time'])
stations_df = pd.read_csv(stations_path)

San_Fancisco_stations = stations_df[stations_df['city'] == 'San Francisco']['id'].unique()

In [9]:
res_df = pd.read_csv("../../Results/Classification_patterns/Classifier_results.csv")

In [ ]:
interval = 15
window_width = 3
split_date = "2015-03-09 09:30:00"
time_zone = None

tot = 0
tot_na = 0

for station in San_Fancisco_stations:
    single_station_df = status_df[status_df['station_id']==station][['time', 'bikes_available', "docks_available"]]
    single_station_df = single_station_df.resample(f"{interval}T", on = 'time').mean().reset_index()
    count = len(single_station_df.loc[single_station_df['time'] >= split_date])
    count_na = count - len(single_station_df.loc[single_station_df['time'] >= split_date].dropna())
    tot += count
    tot_na += count_na
    print(count, count_na)
print("Total values:\t\t", tot)
print("Total null values:\t", tot_na)
print("Total no null:\t\t", tot-tot_na)

In [3]:
interval = 15
window_width = 3
split_date = "2015-03-09 09:30:00"
time_zone = '17-18'
zone_str = f"_{time_zone}" if time_zone != None else ""

if time_zone != None:
        start_h, end_h = time_zone.split('-')
        status_df = status_df[(status_df.time.dt.hour >= int(start_h)) & (status_df.time.dt.hour < int(end_h))]

In [4]:
for reference_station in San_Fancisco_stations:
    merged_df = pd.DataFrame()
    for current_station in San_Fancisco_stations:
        new_df = pd.DataFrame()
        single_station_df = status_df[status_df['station_id']==current_station][['time', 'bikes_available', "docks_available"]]
        windowing_df =  single_station_df.resample(f"{interval}T", on = 'time').mean()
        
        #resetto l'indice
        windowing_df  = windowing_df.reset_index()
                 
        new_df['time'] = windowing_df['time']
        
        #inserisco le colonne 
        for i in reversed(range(window_width)):
            new_df[f'docks_av_{current_station}_T{i}'] =  windowing_df['docks_available'].shift(periods=i)
            new_df[f'bikes_av_{current_station}_T{i}'] =  windowing_df['bikes_available'].shift(periods=i)
        
        #se la stazione corrente è quella di riferimento, inserisco la colonna "status", shiftata indietro di 1 
        # slot temporale in modo che indichi lo stato denna stazione nello slot temporale successivo a quello di riferimento.
        # Questa colonna fungerà da label
        if (current_station == reference_station):
            windowing_df["status"] = np.where(windowing_df["docks_available"] <= 2, "QP", "N")
            new_df["status"] =  windowing_df['status'].shift(periods=-1)
            
        if len(merged_df) == 0:
            merged_df = new_df.copy(deep=True)
            continue
        merged_df = pd.merge(merged_df, new_df, how='outer', on='time')
    
    #Rimuovo tutti i record con valori mancanti
    merged_df = merged_df.dropna()
    
    if not os.path.exists(f'../../Datasets/{interval}_{window_width}'):
        os.makedirs(f"../../Datasets/{interval}_{window_width}")

    #splitto in train e test set
    train_df = merged_df.loc[merged_df['time'] < split_date].copy()
    test_df = merged_df.loc[merged_df['time'] >= split_date].copy()
    
    #rimuovo il tempo in quanto non è utile ai fini della classificazione
    train_df.drop(columns=["time"], inplace=True) 
    test_df.drop(columns=["time"], inplace=True) 
    
    #salvo la versione senza l'informazione sul tempo
    train_df.to_csv(f"../../Datasets/{interval}_{window_width}/station{reference_station}{zone_str}_train.csv", index=False)
    test_df.to_csv(f"../../Datasets/{interval}_{window_width}/station{reference_station}{zone_str}_test.csv", index=False)

In [8]:
classifiers = {
    'DecisionTree': tree.DecisionTreeClassifier(random_state=42),   
    'RandomForest': ensemble.RandomForestClassifier(random_state=42, max_features='sqrt'),
    'XGBoost': xgb.XGBClassifier()
}

In [9]:
start = time.time()

for reference_station in San_Fancisco_stations:

    train_df = pd.read_csv(f"../../Datasets/{interval}_{window_width}/station{reference_station}{zone_str}_train.csv")
    
    y = train_df['status']
    X = train_df.drop(columns=['status'])
    
    #TRAINING
    for classifier in classifiers.keys():
        model = classifiers[classifier].fit(X, y)
        
        #salvo il modello
        if not os.path.exists(f'../../Results/Other_classifiers/Trained_models/{classifier}/{interval}_{window_width}'):
            os.makedirs(f"../../Results/Other_classifiers/Trained_models/{classifier}/{interval}_{window_width}")
        dump(model, f'../../Results/Other_classifiers/Trained_models/{classifier}/{interval}_{window_width}/station{reference_station}{zone_str}_trained_model.joblib')

end = time.time()
print(f'The time of execution is: {end-start} s')

The time of execution is: 1088.3197820186615 s


In [ ]:
result_matrix = []

if not os.path.exists('../../Results/Other_classifiers/Evaluation'):
    os.makedirs('../../Results/Other_classifiers/Evaluation')

for classifier in classifiers.keys():
    
    file = open(f'../../Results/Other_classifiers/Evaluation/{classifier}_{interval}_{window_width}{zone_str}.txt', "w")
    file.write(f'TESING RESULTS FOR {classifier} CLASSIFIER:\n\n')

    tot_fp = 0
    tot_tp = 0
    tot_fn = 0
    tot_tn = 0
    
    for station_id in San_Fancisco_stations:
        model = load(f'../../Results/Other_classifiers/Trained_models/{classifier}/{interval}_{window_width}/station{station_id}{zone_str}_trained_model.joblib')

        test_df = pd.read_csv(f'../../Datasets/{interval}_{window_width}/station{station_id}{zone_str}_test.csv')
        y_test = test_df['status']
        X_test = test_df.drop(columns=['status'])    

        prediction = model.predict(X_test)
        cm = confusion_matrix(y_test, prediction)

        str_= f'{classifier} FOR STATION {station_id}' + '\n'
        str_ += f'Confusion matrix:' + '\n'
        str_ += str(cm) + '\n'

        tn, fp, fn, tp = cm.ravel()
        str_+= f'tp={tp}, fn={fn}, fp={fp}, tn={tn}' +'\n'

        test_accuracy = (tn + tp) / (tn + fp + fn + tp)
        test_recall = (tp) / (tp + fn)
        test_precision = (tp) / (tp + fp)
        test_f1 = 2*test_precision*test_recall/(test_precision+test_recall)

        str_+= f'accuracy={test_accuracy}; recall={test_recall}; precision={test_precision}; F1={test_f1}' +'\n\n'
        str_+= "-"*10 +'\n\n'

        tot_fp += fp
        tot_tp += tp
        tot_fn += fn
        tot_tn += tn
        
        file.write(str_)

    avg_accuracy = (tot_tn + tot_tp) / (tot_tn + tot_fp + tot_fn + tot_tp)
    avg_recall = (tot_tp) / (tot_tp + tot_fn)
    avg_precision = (tot_tp) / (tot_tp + tot_fp)
    avg_f1 = 2*avg_precision*avg_recall/(avg_precision+avg_recall)
    result_matrix.append([avg_accuracy, avg_recall, avg_precision, avg_f1])
    
    avg_str = f"AVERAGE VALUES FOR {classifier}: accuracy={avg_accuracy}; recall={avg_recall}; precision={avg_precision}; F1={avg_f1}"

    file.write(avg_str)
    file.close()
    
results_table = pd.DataFrame(result_matrix, columns=['avg_accuracy', 'avg_recall', 'avg_precision', 'avg_F1'], index=pd.Index(classifiers.keys()))
results_table.to_csv(f"../../Results/Other_classifiers/Evaluation/Overall_results_{interval}_{window_width}{zone_str}.csv")